## This notebook contains code to use raw whisper model using various methods

### 1. Using Whisper Library

In [19]:
_MODEL_PATH = '../models/'
_MODEL_NAME = 'large-v3'
_MODEL_NAME_TORCH = 'whisper-large-v3'
_AUDIO_PATH = './audio_samples/test.wav'


In [1]:
from cloned_repos.whisper.whisper import load_model, load_audio, pad_or_trim, log_mel_spectrogram, DecodingOptions, decode


In [3]:
model = load_model(
    name=_MODEL_NAME,
    download_root=_MODEL_PATH,
    device='cuda:0',
    in_memory=False,
)


In [4]:
audio = load_audio(_AUDIO_PATH)
audio = pad_or_trim(audio)
mel = log_mel_spectrogram(audio, n_mels=128).to(model.device)


In [8]:
options = DecodingOptions(language='en', temperature=0)
result = decode(model, mel, options)
result.text


'Hello world, this is a sample test for the python post request module to the whisper API hosted on the jarvis server. Now I am going to speak in Hindi. So, I have opened my whatsapp and I have messaged a person that he will participate in the event. Thank you very much.'

### 2. Using Transformers library

In [22]:
# _MODEL_PATH = '../models/whisper-large-v3/'
_MODEL_PATH = '../models/whisper-base/'
_AUDIO_PATH = './audio_samples/cwh.m4a'
LANGUAGE = 'en'


In [2]:
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperConfig,
)
import torch
import ffmpeg
import torch
import torch.nn.functional as F
import numpy as np
import time


In [3]:
# load_audio and pad_or_trim functions
SAMPLE_RATE = 16000
CHUNK_LENGTH = 30  # 30-second chunks
N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000 samples in a 30-second chunk


In [4]:
# audio = whisper.load_audio('test.wav')
def load_audio(file: str, sr: int = SAMPLE_RATE, start_time: int = 0, dtype=np.float16):
    """
    Load an audio file into a numpy array at the specified sampling rate.
    """
    try:
        # This launches a subprocess to decode audio while down-mixing and resampling as necessary.
        # Requires the ffmpeg CLI and `ffmpeg-python` package to be installed.
        out, _ = (
            ffmpeg.input(file, ss=start_time, threads=0)
            .output("-", format="s16le", acodec="pcm_s16le", ac=1, ar=sr)
            .run(cmd=["ffmpeg", "-nostdin"], capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

    # return np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0
    return np.frombuffer(out, np.int16).flatten().astype(dtype) / 32768.0


In [5]:
# audio = whisper.pad_or_trim(audio)
def pad_or_trim(array, length: int = N_SAMPLES, *, axis: int = -1):
    """
    Pad or trim the audio array to N_SAMPLES, as expected by the encoder.
    """
    if torch.is_tensor(array):
        if array.shape[axis] > length:
            array = array.index_select(
                dim=axis, index=torch.arange(length, device=array.device)
            )

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = F.pad(array, [pad for sizes in pad_widths[::-1] for pad in sizes])
    else:
        if array.shape[axis] > length:
            array = array.take(indices=range(length), axis=axis)

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = np.pad(array, pad_widths)

    return array


In [23]:
processor = WhisperProcessor.from_pretrained(_MODEL_PATH)
tokenizer = processor.tokenizer

config = WhisperConfig.from_pretrained(_MODEL_PATH)

model = WhisperForConditionalGeneration(config=config).from_pretrained(
    _MODEL_PATH,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
    # use_flash_attention_2=True
).to('cuda:0')
# model.to_bettertransformer()
model.eval()


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=False)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          

In [21]:
start_time = time.perf_counter()
audio = load_audio(_AUDIO_PATH)
audio = pad_or_trim(audio)
end_time = time.perf_counter()

print(f'Inference time: {end_time - start_time}')
from IPython.display import Audio
Audio(audio, rate=SAMPLE_RATE)


Inference time: 2.2440568339989113


In [24]:
input_features = processor(audio, sampling_rate=SAMPLE_RATE, return_tensors="pt").input_features.to('cuda:0')
input_features.shape


torch.Size([1, 80, 3000])

In [25]:
start_time = time.perf_counter()
with torch.no_grad():
    predicted_ids = model.generate(
        input_features,
        # num_beams=4,
        language=LANGUAGE,
        task="transcribe",
        use_cache=True,
        is_multilingual=True,
        return_timestamps=True,
    )
transcription = tokenizer.batch_decode(
    predicted_ids, 
    skip_special_tokens=True,
)[0]
end_time = time.perf_counter()

print(f'Inference time: {end_time - start_time}')
print(transcription.strip())


Inference time: 0.9397452060002252
Guys, the time has come here. In C++, most importantly, we will discuss the important topic. The topic is called Object oriented programming. What is Object oriented programming? Let's talk about it and see what that terminology is. We should always keep in mind the terms of the object oriented programming. First of all, I want to start what is Object oriented programming? First of all, Object oriented programming is not a new programming language.
